In [1]:
import pandas as pd
import numpy as np
import matplotlib as mtpl
import matplotlib.pyplot as plt

font = {'family' : 'serif','weight' : 'ultralight','size'   : 14}
mtpl.rc('font', **font)

In [13]:
def Smooth121(x):
    signal = x
    length = len(signal)
    output = np.zeros(length-2)
    coef= np.array([1,2,1])
    for i in range(length - 2):
        output[i]= np.sum(signal[i:i+3] * coef / 4)
    return output

def Haar(ix,smooth=True,bins=20):
    x=np.array(lis[ix][lis[ix].columns[1]])
    t=np.array(lis[ix][lis[ix].columns[0]])

    #Cálculo deltas
    deltas_t=[]
    epsilons=[]
    for i in range(1,int(len(t)/2)+1):
        deltas_t.append(t[i:]-t[:-i])
    for i in range(1,len(deltas_t)+1):
        epsilons.append(deltas_t[i-1][:-i]/(deltas_t[i-1][:-i]+deltas_t[i-1][i:]))
    x=x[:-1];t=t[:-1]
    
    #Cálculo fluctuaciones
    Hs=[]
    delta_t=[]
    ep_min=0.25
    ep_max=1-ep_min
    calib=2
    counter=0
    
    for H in range(2,len(x)-1,2):
        for start in range(len(x)-H-1):
            int1=np.sum(x[start:start+int(H/2)]*deltas_t[0][start:start+int(H/2)]/deltas_t[int(H/2)-1][start])
            int2=np.sum(x[start+int(H/2):start+H]*deltas_t[0][start+int(H/2):start+H]/deltas_t[int(H/2)-1][start+int(H/2)])
            counter+=1
            if ep_min<epsilons[int(H/2)-1][start] and epsilons[int(H/2)-1][start]<ep_max:
                Hs.append((calib*(int2-int1))**2)
                delta_t.append(deltas_t[int(H/2)-1][start]+deltas_t[int(H/2)-1][start+int(H/2)])
    
    #Paso a dataframes
    data_df=pd.DataFrame(data={'delta t':delta_t , 'Hs': Hs})
    data_sorted=data_df.sort_values('delta t',axis=0).reset_index(drop=True)

    max_t=max(np.log10(data_sorted['delta t'])); min_t=min(np.log10(data_sorted['delta t']))
    n_bins=int(((max_t)-(min_t))*bins); rango_int=(max_t-min_t)/n_bins; time=np.array([]); ave_values=np.array([]); upper=np.array([]); lower=np.array([]);
    for i in range(n_bins):
        interval=data_sorted[(i*rango_int+min_t<=np.log10(data_sorted['delta t']))&(np.log10(data_sorted['delta t'])<min_t+(i+1)*rango_int)]
        time=np.append(time,((i*rango_int+min_t)+((i+1)*rango_int+min_t))/2)
        ave_values=np.append(ave_values,np.sqrt(interval.mean()[1]))        ## Tomo el sqrt del <[2*(x1-x2)]^2> ...
        upper=np.append(upper,np.sqrt(interval.mean()[1]+interval.std()[1]/np.sqrt(len(interval))))
        lower=np.append(lower,np.sqrt(interval.mean()[1]-interval.std()[1]/np.sqrt(len(interval))))
        
    if smooth==False:
        smooth_val=ave_values
    if smooth==True:
        smooth_val=Smooth121(ave_values)
        upper=Smooth121(upper)
        lower=Smooth121(lower)
        time=time[1:-1]
    val_mask = np.isfinite(smooth_val)
    
    time=time[val_mask] #??
    smooth_val=smooth_val[val_mask] #??
    upper=upper[val_mask]
    lower=lower[val_mask]

    return time,smooth_val,upper,lower


In [6]:
dust_file='EDC_DustFlux_25yr.xlsx'

id_columns=['Name','Data id','Latitud','Longitud','Age units','Data units','Data length']
id_data=pd.read_excel(dust_file,sheet_name=0,usecols=id_columns)
df_data=pd.read_excel(dust_file,sheet_name=1,skiprows=1)

#SEPARAR COLUMNAS
columns=df_data.size/len(df_data) ;lis=[];new_length=np.array([])

for i in range(int(columns)):
    if (i+1)%2==1: 
        dupla=df_data[[df_data.columns[i],df_data.columns[i+1]]].dropna() 
        dupla.index=[j for j in range(0, len(dupla))] 
        lis.append(dupla)
        new_length=np.append(new_length,np.shape(dupla)[0])

In [15]:
[time1,val1,upper1,lower1]=Haar(0)

In [34]:
t=pd.DataFrame(time1,columns=['Time'])
v=pd.DataFrame(val1,columns=['Values'])
u=pd.DataFrame(upper1,columns=['Upper'])
l=pd.DataFrame(lower1,columns=['Lower'])

haar_data=pd.concat([t,v,u,l],axis=1)

In [35]:
haar_data.to_excel('Haar4cluster results1.xlsx')